# Project 1 - Data Engineering
## 2 Data Exploration and Analysis
This notebook is for the implementation of task "2 Data Exploration and Analysis", as listed in the **Project Instructions**.

<div class="alert alert-success">
<b>Overview:</b><br>
Data Exploration and Analysis consists of the following parts worth 70 points:
<ul>
    <li><b>E1 - Obtain and Scrub</b> (15 points)</li>
    <li><b>E2 - Exploratory data analysis (EDA) </b>(20 points)</li>
    <li><b>E3 - Formulate hypotheses </b>(25 points)</li>
    <li><b>Follow the guidelines for Data Exploration and Analysis below </b>(10 points)</li>
</ul></div>

<div class="alert alert-success">
<b>Guidelines for Data Exploration and Analysis:</b><br>
    <ol>
        <li>Use a single Jupyer notebook for your project.</li>
        <li>Use only Python-code for your project.</li>
        <li>The use of automatic and semi-automatic data analyis tools is not allowed (e. g., PandasGUI, D-Tale, Mito, etc.). Only use packages we used in the coded lectures.</li>
        <li>Export your environment for submission as 'prj01-environment.txt'.</li>
        <li>Upload your resulting work as a zip file containing only a single jupyter notebook and required files to run the notebook. All cell outputs and <b>figures must display in jupyter lab</b>. (Test this, in particular when you use another environment like VS Code.)</li>
        <li>All code cells in your notebook must be runnable without errors or warnings (e. g., deprecated functions). Each error/warning subtracts -2 points (up to the full 10 points for following the  guidelines).</li>
        <li>Use only relative paths in your project.</li>
        <li>Avoid (excessive) code duplication.</li>
        <li>Avoid loops iterating over pandas objects (Series, DataFrames). Explicitly justify each exception via a comment. </li>
        <li>All coded steps in your analysis must be commented.</li>
        <li>Keep your code as well as outputs short, precise and readable. Each long or unnecessary output subtracts -2 points (up to the full 10 points for following the project guidelines).</li>
    </ol>
    <b>Late submissions are not accepted and earn you 0 points on the python project. </b>
</div>

Explicitly list which notebook toolset was used (jupyter lab/jupyter notebook/VS Code/etc.)

**here**: MY_TOOL, MY_BROWSER

Explicitly and clearly state the chosen dataset number and title:
### Bevölkerung ODÖ Hunde
#### Hundebestand seit 2002 - Bezirke Wien --> contains info about dog density per districts over years
#### Hunde pro Bezirk Wien --> contains info about dog breeds count per district in 2024
#### Hunderassen Wien --> contains info about dog breeds count per district 2012 - 2017


## E1 - Obtain and Scrub

### Obtain
Download the dataset and understand both:
- format: wide vs. long, separaters, decimal points, encoding, etc., and
- content: what variables are in the columns, what is their meaning?
To this end, identify and download metadata such as headers, category listings, explanatory reports, etc.
### Scrub
The aim of scrub is to create a clean version of the data for further analysis.
- Load the dataset and take care of dtypes (dates, numbers, categories, etc.). Justify why you don't load/use specific columns.
- Check for footnotes or any other notifications on special cell content, such as time series breaks. Follow up that information, and document your decision how to deal with it. Remember: A homework contained such info in the cell "76.1 b". The metadata defines what that "b" stands for.
- Choose an appropriate (Multi-)Index.
- Identify:
    1. missing values and get row and column overviews. Use graphical and/or numeric approaches. Once identified, handle missing values according to column type, time series property and data set size.
    2. duplicates (justify the used column subset). Remove duplicates - if any - and inspect what you removed.
- Transform to shape (tidy vs. wide) best suited for further analysis.
- Export the clean data to a file for inspection with an external data browser (e.g., MS Excel).
- Provide an overview of the clean dataset:
    1. show the dtypes
    2. quantitative column descriptions:
        1. categorical columns: number of unique values, counts
        2. numeric columns: range and median

In [ ]:
# Importing necessary packages:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps
import geopandas as gpd
import json # to read in geojson
from matplotlib.lines import Line2D # for custom legend
%matplotlib inline

## 1) Obtain the datasets and scrub data


#### 1) Dataset 1
Number of dogs (absolute and per 1,000 inhabitants) since 2002 - districts of Vienna
 	* NUTS | NUTS2-Region (Bundesland)
  * DISTRICT_CODE | Gemeindebezirkskennzahl (Schema: 9BBZZ, BB=Nummer Bezirk, ZZ=00)
  * SUB_DISTRICT_CODE | Zählbezirkskennzahl gemäß Stadt Wien (Schema: 9BBZZ, 9=Kennzeichnung Wien, BB=Nummer Bezirk, ZZ=Nummer Zählbezirk, **ZZ=99 bei fehlender Zählbezirkskennzeichnung**)
  * REF_YEAR | Referenzjahr
  * REF_DATE | Referenzdatum
  * DOG_VALUE | Anzahl der Hunde (absolut)
  * DOG_DENSITY | Anzahl der Hunde pro 1.000 EinwohnerInnen

Source: https://www.data.gv.at/katalog/dataset/1a48e78a-c660-41e0-ac32-8b675c0ea39c#additional-info

   TODO: add more description?

##### Obtain

In [ ]:
# Import the first data set
dogs_2002 = pd.read_csv("vie-bez-biz-spo-dog-2002f.csv", sep=';', skiprows=1, dtype={"DOG_VALUE": "object", "DOG_DENSITY": "object"})
print("Dataframe shape: ", dogs_2002.shape)
dogs_2002.head(5)

* The separator is ';'.
* The encoding is 'utf-8'.
* The original shape is (528, 17) - but many columns full of NaNs (later: cleaned up data => shape=(528, 7)).
* Long format:  
  * Contains entries for years by district of overall dog density sorted by years.
  * each row represents a single observation of district by year and dog breeds
* The first row contains the csv title. It is dropped and the second row is used for column titles.
* First column 'NUTS' (= identification and classification system for european states) contains always the same value, since 'AT13' stands for Vienna (and we only have data for Vienna in this dataset). We could drop it, but it also doesn't bother us too much right now
* DISTRICT_CODE '90000' stands for all districts combined
* Need to import DOG_VALUE as objects because the '.' in some values does not allow conversion to integer right now. Without specifying it, the values are recognised as floats.
* We also import DOG_DENSITY as objects because we need to convert the ',' to '.' to allow conversion to floats afterwards


##### Scrub

In [ ]:
# check columns and their missing data ratio
((dogs_2002.isnull().sum() / len(dogs_2002)) * 100).sort_values(ascending=False)

In [ ]:
# drop all cols with NaN --> lots because of multiple 'unnecessary' seperators ';' in last columns
# other columns have no missing values
dogs_2002 = dogs_2002.dropna(axis=1, how='any')
print("Dataframe shape: ", dogs_2002.shape)

In [ ]:
print(dogs_2002.dtypes)

Remove '.' in DOG_VALUE and convert to integer.

Change DOG_DENSITY to use '.' instead of ',' and the format to float to be able to calculate with it, if needed.

Change REF_DATE to format datetime since its currently int64.

In [ ]:
dogs_2002['DOG_VALUE'] = dogs_2002['DOG_VALUE'].str.replace('.', '').astype(int)
dogs_2002['DOG_DENSITY'] = dogs_2002['DOG_DENSITY'].str.replace(',', '.').astype(float)
dogs_2002['REF_DATE'] = pd.to_datetime(dogs_2002['REF_DATE'].astype(str), format='%Y%m%d')

In [ ]:
print(dogs_2002.dtypes)
dogs_2002.head(2)

* District Code and Sub District Code are the same --> removing Sub District Code
* add Postal_CODE column for better comparison between datasets

In [ ]:
# only execute if not already done (avoids errors)
if 'SUB_DISTRICT_CODE' in dogs_2002.columns:
    print(np.unique([dogs_2002['DISTRICT_CODE'] == dogs_2002['SUB_DISTRICT_CODE']], return_counts=True)) # check that columns are actually the same
    dogs_2002.drop(columns=['SUB_DISTRICT_CODE'], inplace=True)
dogs_2002["Postal_CODE"] = "1" + dogs_2002['DISTRICT_CODE'].astype(str).str[1:3] + "0"
dogs_2002.head(2) # Postal_CODE=1000 represents Vienna in total

Check for duplicates --> looks good:

In [ ]:
dogs_2002.duplicated(subset=['DISTRICT_CODE', 'REF_YEAR']).any()

#### Dataset 2
  Number of registered dogs in Vienna per district and dog breed in the year 2024.

  * NUTS1 NUTS2 NUTS3: same as before, but NUTS3 in addition: provides district information for Vienna
  * DISTRICT_CODE: same structure as before
  * SUB_DISTRICT_CODE
  * Postal_CODE
  * Dog Breed
  * Anzahl
  * Ref_Date

Source: https://www.data.gv.at/katalog/dataset/71edef44-9d6c-4042-ab71-7207dc930ba7#additional-info

##### Obtain

In [ ]:
# Import dataset 2
dog_types_2024 = pd.read_csv("hunde-wien.csv", sep=';', encoding='latin-1', skiprows=1, dtype={"Anzahl": "object"})

print("Dataframe shape: ", dog_types_2024.shape)
dog_types_2024.head(5)

* The separator is ';'
* The encoding is 'latin-1'.
* The shape is (11117, 9).
* Mixed breeds are also included, like 'Hovawart / Golden Retriever'.
* Format is long:
  * dog breeds can appear multiple times in breed-column. This is the case if at least two dogs of a breed are registered in different districts. If there is no dog of a specific breed registered in a distric, it is not listed in the dataset for this district.
  * Every district and breed combination has a single count value
* We import 'Anzahl'-column as objects, because there is a value that contains a '.' (Point). Therefore, it would be recognised as a float even though these values should be all integers.

##### Scrub

In [ ]:
# check columns and their missing data ratio => no missing data
((dog_types_2024.isnull().sum() / len(dog_types_2024)) * 100).sort_values(ascending=False)

In [ ]:
dog_types_2024.dtypes.to_frame().T

Looks like some columns only have 1 unique value => NUTS1-3, SUB_DISTRICT_CODE and Ref_Date to be exact. Also, no district is missing, which is good.

In [ ]:
dog_types_2024.nunique()

In [ ]:
print("DISTRICT_CODE:", dog_types_2024["DISTRICT_CODE"].unique())
print("Postal_CODE:", dog_types_2024["Postal_CODE"].unique())

Drop NUTS1, NUTS2, NUTS3 and SUB_DISTRICT CODE. Don't want to drop Ref_Date yet, maybe could be used for some insights with the above data frame.

In [ ]:
dog_types_2024 = dog_types_2024.drop(['NUTS1', 'NUTS2', 'NUTS3', 'SUB_DISTRICT_CODE'], axis=1)
dog_types_2024['Ref_Date'] = pd.to_datetime(dog_types_2024['Ref_Date'].astype(str), format='%Y%m%d')
dog_types_2024.dtypes.to_frame().T

The 'Anzahl'-column contains one value that currently cannot be converted to an integer without losing information. It seems that there are 1012 dogs of mixed breed 'Unbekannt / Mischling', which would be recognised as a float with decimal places:

In [ ]:
dog_types_2024[dog_types_2024["Anzahl"].str.contains(".", regex=False)]

In [ ]:
dog_types_2024["Anzahl"] = dog_types_2024["Anzahl"].str.replace('.','', regex=False).astype(int)
print(dog_types_2024.dtypes) # we could also change dtype for 'Dog Breed' to 'category', but 3500 categories are a bit much. We leave it for now
dog_types_2024.head(2)

Check if some Dog Breeds have duplicates per district --> looks good.

In [ ]:
dog_types_2024.duplicated(subset=['Dog Breed', 'Postal_CODE']).any()

Check if there are unknown Dog Breeds:
  * There appear to be in total 50 entries in the dataset with at least partly unknown Dog Breeds.
  * There are in total 8200 dogs with an at least partly uknown dog breed.
  * The dataset has only 6 unique values in the 'Dog Breed'-column that contain an unknown dog breed.
  * They are kept for the moment.

In [ ]:
def check_for_unknown_dog_breeds(df: pd.DataFrame):
    bool_unknown = df['Dog Breed'].str.contains('Unbekannt', case=False)
    print("Total number of entries with unknown dog breed:", bool_unknown.sum())
    print("Total number of dogs with at least partly unknown dog breed:", df[bool_unknown]["Anzahl"].sum())
    tmp_array = df[bool_unknown]['Dog Breed'].unique()
    print("Number of unknown dog breeds:", len(tmp_array))
    print("Unknown dog breeds:", tmp_array)

check_for_unknown_dog_breeds(dog_types_2024)

In [ ]:
dog_types_2024.head()

We could also drop the postal_code column, since it basically provides the same data as the district_code column:

In [ ]:
(dog_types_2024["DISTRICT_CODE"].astype(str).str[1:3] == dog_types_2024["Postal_CODE"].astype(str).str[1:3]).unique() # compare 2-digit district code

#### Dataset 3
Number of registered dogs in Vienna per district and dog breed in the years 2012 to 2017. 

The layout of the dataset is basically the same as the one before, we only have data for multiple years.

  * NUTS1: AT1
  * NUTS2: AT13
  * NUTS3: AT113
  * DISTRICT_CODE: Bezirke, Format 9BB00
  * SUB_DISTRICT_CODE: Zählbezirk, leer
  * Postal_CODE: Postleitzahl, Format 1BB0
  * Dog Breed: Hunderasse
  * Anzahl: Anzahl der jeweiligen Hunderasse
  * Ref_Date: Jahr

Source: https://www.data.gv.at/katalog/dataset/31f526f5-2cf2-49f8-a63d-0c63a0f87a3e

In [ ]:
# Import dataset 3
dog_types_2012 = pd.read_csv("hunde-vie.csv", sep=';', encoding='latin-1', skiprows=1, dtype={"Anzahl": "object"})

print("Dataframe shape: ", dog_types_2012.shape)
dog_types_2012.head(5)

* The separator is ';'
* The encoding is 'latin-1'.
* The shape is (33793, 9).
* Format is long:
  * dog breeds can appear multiple times in breed-column. This is the case if at least two dogs of a breed are registered in different districts or in different years. If there is no dog of a specific breed registered in a distric, it is not listed in the dataset for this district.
  * Every district, ref_date and breed combination has a single count value
* We import the 'Anzahl'-column as objects and process them the same way as in the previous dataset.


Repeat steps as with dataframe 2, since they have the same format.

In [ ]:
# check columns and their missing data ratio => no missing data
((dog_types_2012.isnull().sum() / len(dog_types_2012)) * 100).sort_values(ascending=False)

Looks like some columns only have 1 unique value => NUTS1-3 and SUB_DISTRICT_CODE to be exact. Also, no district is missing, which is good. We have 6 different years in the dataset:

In [ ]:
dog_types_2012.nunique()

Column Ref_Date contains more entries than above, lets see which years are included: 2012 to 2017

In [ ]:
dog_types_2012['Ref_Date'].value_counts()

In [ ]:
print(dog_types_2012.groupby("Ref_Date")["DISTRICT_CODE"].nunique()) # no districts missing in each year
print(dog_types_2012.groupby("Ref_Date")["Postal_CODE"].nunique())

In [ ]:
dog_types_2012.dtypes.to_frame().T

'Anzahl' can be converted to integer this time without any additional steps:

In [ ]:
print(dog_types_2012[dog_types_2012["Anzahl"].str.contains(r"[\.\,]", regex=True)]) # no entry with '.' or ','
dog_types_2012["Anzahl"] = dog_types_2012["Anzahl"].astype(int)

Change Ref_Date from int64 to date (important: format is now different to before: yyyy**dd**mm, not yyyymm**dd**):

In [ ]:
dog_types_2012 = dog_types_2012.drop(['NUTS1', 'NUTS2', 'NUTS3', 'SUB_DISTRICT_CODE'], axis=1)
dog_types_2012['Ref_Date'] = pd.to_datetime(dog_types_2012['Ref_Date'].astype(str), format='%Y%d%m') # convert int to real datetime
print(dog_types_2012.dtypes.to_frame().T)

Check for duplicates and unkown dog breeds --> has only the unknown dog breed 'Unbekannt' and no duplicates too.
* Unkown can stay for now, even though their number is very high with 98.707 dogs (even if we consider that the dataset contains multiple years of data).



In [ ]:
dog_types_2012.duplicated(subset=['Dog Breed', 'Postal_CODE', 'Ref_Date']).any()

In [ ]:
check_for_unknown_dog_breeds(dog_types_2012)

### Merging Dataset 2 and 3 as they contain the same info for different years

In [ ]:
print(dog_types_2024.dtypes.to_frame().T)
print(dog_types_2012.dtypes.to_frame().T)

In [ ]:
combined_dog_types = pd.concat([dog_types_2012, dog_types_2024]) # dataframes have same columns => concat
print("Shape:", combined_dog_types.shape)

In [ ]:
combined_dog_types_by_year_and_district = combined_dog_types.pivot_table(index=['Ref_Date', 'Postal_CODE'], columns='Dog Breed', values='Anzahl', fill_value=0).sort_values(by='Ref_Date', ascending=False)
combined_dog_types_by_year_and_district.head(2)

Sanity Check: 

Values are the same (apart from 0) --> looks good. 

In [ ]:
unique_values = []
unique_values_single = []
for col in combined_dog_types_by_year_and_district.columns:
    unique_values.extend(combined_dog_types_by_year_and_district[col].unique())

unique_values_single.extend(dog_types_2012['Anzahl'].unique())
unique_values_single.extend(dog_types_2024['Anzahl'].unique())
unique_values_single = set(unique_values_single)
unique_values = set(unique_values)
differences = unique_values.symmetric_difference(unique_values_single)
print(differences)

## E2 - Exploratory data analysis (EDA)
Use the clean dataset and understand and explore the relationships in the data (numerical, visual, statistical). This includes at least but is not limited to:
- A comprehensive textual description of meaning for relevant fields in the dataset
- Statistical/numerical descriptions and visualization techniques we learned in the course inluding correlations, distributions and groupings of variing degrees.
- Checks for data quality, e. g., completeness, plausibility, outliers
- Handling any identified problems with the data
- If necessary, use additional data wrangling in line with your EDA and only keep what's necessary for the following steps of your analysis with appropriate data granularity and form ("tidy data")

Don't:
- test/prove hypotheses here. EDA should only motivate hypotheses.

Advanced/bonus:
- Depending on your hypothesis you may want to join external data (e.g., merge external highest education level to existing vaccination data) for additional insights.

Explain all steps taken and your thinking why you deem them necessary.

In [ ]:
# No missing data - TODO: heatmaps still necessary?


BOXPLOTS

In [ ]:
# dogs_2002: Check which DOG_VALUEs are outliers for each year
# TODO: add comments
df_dogs_2002_boxplots = dogs_2002[(dogs_2002["Postal_CODE"] != "1000")].sort_values(by=["REF_YEAR", "Postal_CODE"], ascending=[True, True]) # sort for year first, so we can correctly append the isOutlier column
array_is_outlier = np.array([], dtype=bool)
whis_ratio = 1.5
for year in range(df_dogs_2002_boxplots["REF_YEAR"].min(), df_dogs_2002_boxplots["REF_YEAR"].max() + 1): # using loop across years to calculate quartiles only once per year
    tmp_filtered_df = df_dogs_2002_boxplots[df_dogs_2002_boxplots["REF_YEAR"] == year] # save filtered df because we need it multiple times
    first_quartile, median, third_quartile = tmp_filtered_df["DOG_VALUE"].quantile([0.25, 0.5, 0.75])
    len_whisk = (third_quartile - first_quartile) * whis_ratio
    upper_whisk_limit = third_quartile + len_whisk
    lower_whisk_limit = first_quartile - len_whisk
    array_is_outlier = np.append(array_is_outlier, 
                                 np.where( (tmp_filtered_df["DOG_VALUE"] > upper_whisk_limit) | (tmp_filtered_df["DOG_VALUE"] < lower_whisk_limit), # if outlier, set True
                                          True, False))
df_dogs_2002_boxplots["Outlier"] = array_is_outlier

palette = sns.color_palette("Set2")[0:2] # get two colors from a sns palette
g = sns.boxplot(df_dogs_2002_boxplots, y="DOG_VALUE", x='REF_YEAR', width=.5, whis = whis_ratio, legend=False)
g = sns.swarmplot(df_dogs_2002_boxplots, y="DOG_VALUE", x='REF_YEAR', size=3.25, hue='Outlier', palette=palette, legend=False)
outlier_districts = df_dogs_2002_boxplots[df_dogs_2002_boxplots["Outlier"]]["Postal_CODE"].unique()
plt.legend(title=None, labels=[f"Outlier districts: {', '.join(outlier_districts)}"], handles=[Line2D([0],[0], marker='o', color='w', label='Scatter', markerfacecolor=palette[1], markersize=7)])
sns.despine()    # remove the top and right line in graph
g.figure.set_size_inches(15,6)
g.set(xlabel="Year", ylabel="Absolute number of dogs")
g.set_title("Absolute number of dogs in Vienna per year", loc="left", weight='bold', fontsize=18)
x_location = g.axes.get_xticklabels()[-1].get_position()[0]
for district in outlier_districts:
    y_location = df_dogs_2002_boxplots[(df_dogs_2002_boxplots["REF_YEAR"]==2023) & (df_dogs_2002_boxplots["Postal_CODE"]==district)]["DOG_VALUE"].values[0] + 150
    plt.annotate(text=district, xy=(x_location, y_location))
plt.show()
# TODO: please remove output of this cell (=plot) before commit

BARPLOTS

In [ ]:
# Not so good - Barplot: Dogs per breed in Vienna per year
#df_top10_2012 = dog_types_2012.groupby(["Ref_Date", "Dog Breed"]).agg(count = ("Anzahl", "sum")).sort_values(["count"], ascending=False).groupby(["Ref_Date"]).head(100) # double groupby and agg using sum-function to avoid deprecation warning of other methods
#df_top10_2012 = df_top10_2012.reset_index()
#fig = px.bar(df_top10_2012, x="Ref_Date", y="count", color="Dog Breed", title="Dogs per breed in Vienna per year")
# TODO: leftshift years 1 year
#fig.show()

In [ ]:
# Plot: Absolute number of dogs of the top 30 breeds (across all districts in 2024)
sum_dog_types_2024 = pd.DataFrame(dog_types_2024.groupby("Dog Breed")["Anzahl"].sum())
top30_sum_dog_types_2024 = sum_dog_types_2024.sort_values(by="Anzahl", ascending=False).head(30).reset_index()
bool_unbekannt_top30_2024 = top30_sum_dog_types_2024["Dog Breed"].str.contains("Unbekannt", case=False)

fig, ax = plt.subplots(figsize=(10,7))
sns.barplot(top30_sum_dog_types_2024, x="Anzahl", y="Dog Breed", color="blue", ax=ax)
sns.despine(ax=ax, trim=True)
for i in np.arange(len(top30_sum_dog_types_2024))[bool_unbekannt_top30_2024]:
    ax.patches[i].set_color('orange')
ax.set_title("Absolute number of dogs of the top 30 breeds (across all districts in 2024)\n[Unknown marked in orange]", loc="left")
ax.grid(axis="x")
# TODO: please remove output of this cell (=plot) before commit

In [ ]:
# Barplot: Absolute number of dogs across all districts in Vienna per year (dogs_2002)
fig = px.bar(dogs_2002[dogs_2002["Postal_CODE"] != "1000"], x="REF_YEAR", y="DOG_VALUE", color="Postal_CODE", title="Absolute number of dogs across all districts in Vienna per year (2002-2023)",
             labels={"REF_YEAR": "year","DOG_VALUE": "Absolute number of dogs","Postal_CODE": "postal code"}
            )
fig.update_xaxes(dtick=1)
fig.show()
# TODO: please remove output of this cell (=plot) before commit

In [ ]:
# TODO: delete or find useful plot
df_top10_2024 = dog_types_2024.groupby(["Postal_CODE", "Dog Breed"]).agg(count = ("Anzahl", "sum")).sort_values(["count"], ascending=False).groupby(["Postal_CODE"]).head(10) # double groupby and agg using sum-function to avoid deprecation warning of other methods
df_top10_2024 = df_top10_2024.reset_index()

In [ ]:
# plot: total number of dogs per district and year
# TODO: how to handle jump between 2018 and 2024? Make colors better, maybe continuous color scale?
df_years_district = pd.DataFrame(combined_dog_types.groupby(["Ref_Date", "Postal_CODE"])["Anzahl"].sum().reset_index())
px.line(df_years_district, x="Ref_Date", y="Anzahl", color='Postal_CODE', markers=True)
# TODO: please remove output of this cell (=plot) before commit

In [ ]:
# preprocessing for geo-data plots
df_beznr_district_mapping = pd.read_csv("beznr-district-mapping.csv", sep=',', dtype={"BEZNR": object}) # additional mapping between beznr and district name because geojson contains only name of district
df_beznr_district_mapping.head(2)

In [ ]:
# plot: total number of dogs per district and year
df_years_district["BEZNR"] = df_years_district["Postal_CODE"].astype(str).str[1:3].astype("int32").astype(str)
df_years_district["year"] = df_years_district["Ref_Date"].dt.strftime("%Y").astype("int32")
df_years_district = df_years_district.merge(df_beznr_district_mapping, on="BEZNR", how="left")
with open('vienna_.geojson', 'r', encoding="utf8") as file:
    geo_json = json.load(file)
fig = px.choropleth(df_years_district,
                    geojson=geo_json,
                    color="Anzahl",
                    animation_frame='year',
                    locations="District_Name",
                    featureidkey="properties.name",
                    projection="mercator",
                    color_continuous_scale="deep",
                    title="Total number of dogs per district and year"
                   ) # TODO: maybe use log scaling
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=500,width=500)
fig.show()
# TODO: please remove output of this cell (=plot) before commit

In [ ]:
# plot: absolute number of dogs since 2002
geo_dogs_2002 = dogs_2002.copy()
geo_dogs_2002["BEZNR"] = geo_dogs_2002["DISTRICT_CODE"].astype(str).str[1:3].astype("int32").astype(str)
geo_dogs_2002 = geo_dogs_2002.merge(df_beznr_district_mapping, on="BEZNR", how="left")
fig = px.choropleth(geo_dogs_2002,
                    geojson=geo_json,
                    color=np.log2(geo_dogs_2002["DOG_VALUE"]), # TODO: maybe use log scaling
                    animation_frame='REF_YEAR',
                    locations="District_Name",
                    featureidkey="properties.name",
                    projection="mercator",
                    color_continuous_scale="deep",
                    title="absolute number of dogs since 2002"
                   ) 
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=500,width=500)
fig.update_layout(coloraxis_colorbar=dict(
    title="Absolute number [log2]"#,
    #tickvals=[6,7,8,9],
    #ticktext=["", "10M", "100M", "1B"],
))
fig.show()
# TODO: maybe concentrate on some specific districts
# TODO: please remove output of this cell (=plot) before commit

## E3 - Formulate hypotheses

*Note: Read this section entirely and understand it - every group member.*

A hypothesis is an idea or explanation for something that is based on known facts but has not yet been proved. A hypothesis is a compact, concise statement, such as: "Individuals with higher income have (on average) more offspring.", that will be answered based on facts (the data). https://gradcoach.com/what-is-a-research-hypothesis-or-scientific-hypothesis/

Formulate *N* non-trivial hypotheses, 1 per group member, and regard the following criteria:
- State the hypothesis explicitly in concise language.
- The hypothesis must be **motivated** by either **EDA results** or **literature** (citation in the report needed).
- The hypothesis must refer to **endpoints** that are **testable**. Specifically, the endpoint must be derived from the data.
- Think of real-life use cases/consequences of your results (textual description).
- For each hypothesis explain all executed steps.
- In case of extreme or implausible results check the validity of your data.
- For each hypothesis export the artifacts (figures, tables, etc.) required for the report.
- If you decide to use a statistical test, use it properly. In particular, check the validity and comparability of the samples.

Do not:
- State nebulous, vague hypotheses. These don't contain endpoints and are unclear to test (i.e., answer).
- Use post-hoc hypotheses. Portraying an empirically inspired **post hoc hypothesis as a priori** violates the **falsification principle** crucial for hypothesis-driven (that is, confirmatory) empirical research. Falsification is severe scientific fraud.
- State trivial hypotheses (e.g., hypthesis 2: "Not Hypothesis 1").
- Answer based on "common knowledge".
- Try to **produce positively tested hypotheses**. If a well motivated hypothesis is negative, this is an important finding (see Simpson's Paradox). The value of a tested hypothesis lies in the information or learning it provides.

Example: The homework with Simpson's Paradox. The pooled overall comparison between the genders would be the EDA motivating the hypothesis: "At UC Berkeley the by-department admissions rate for females is lower than for males." It should be tested using samples of department admission rates for the 2 **groups** male and female. No steps of the test should be done in EDA (or a priory to stating the hypothesis). The groups should be compared graphically, e.g., via a stripplot overlayed with a boxplot. The figure should be labelled properly and exported for the report. A (paired) t-test **may** be used (it's optional) to test this hypothesis statistically. For different data (e.g., time series) different approaches may be required. You don't have to use statistical tests, in particular if you don't know what they are doing.


---

### E3-H1: "The prevalence of specific dog breeds in Vienna is strongly influenced by real estate prices."

Rationale: Higher real estate prices may attract certain dog breeds, indicating a potential link between the types of dogs found in high-income areas and local housing costs.

Data Needed: 
- Dog breed data (`hunde-vie.csv` / `hunde-wien.csv`).
- Real estate prices dataset.

Author: Carlos Eduardo Tichy

---

### Supporting Evidence and Insights

While there is no direct study linking **dog breed prevalence** to **real estate prices** specifically in Vienna, several sources suggest that **housing conditions and socio-economic factors** do influence dog breed choices across Europe.

1. **Housing Conditions**: A study noted that housing conditions, such as **living in an apartment or house with a garden**, strongly influence the types of dogs people own. In urban areas with high real estate prices, smaller breeds, like **French Bulldogs or Chihuahuas**, are more common due to space constraints. In contrast, **larger breeds**, such as Golden Retrievers or German Shepherds, tend to be more prevalent in suburban or rural areas where properties are larger and more affordable  .

2. **Socio-Demographic Factors**: Research also highlights that wealthier households or those in areas with **higher disposable income** are more likely to own certain breeds that are more expensive to maintain, such as **purebred dogs**. Similarly, a rise in **single-person households** and more flexible work patterns (like remote work) have made dog ownership more common in urban areas, further influencing breed choices based on lifestyle  .

3. **Popularity and Behavior**: Another study found that certain breeds become popular in specific socio-economic environments due to **behavioral traits** that suit urban living. For example, dogs that are **trainable, less aggressive, or more adaptable to confined spaces** are preferred in cities .

These insights suggest a clear relationship between the environment (including **real estate prices**) and **dog breed selection**, even if specific studies on Vienna are yet to emerge.

---

**Sources**:

1. [The Influence of Breed, Sex, Origin and Housing Conditions on Undesirable Behaviors in Ancient Dog Breeds](https://www.mdpi.com/2076-2615/11/5/1435)
2. [The Rise of Dog Ownership in Europe: A Decade in Review](https://rochsociety.com/research-european-dog-ownership-2010-2022/)
3. [Socio-demographic factors associated with pet ownership amongst adolescents from a UK birth cohort](https://bmcvetres.biomedcentral.com/articles/10.1186/s12917-019-2063-x)
4. [Fashion vs. Function in Cultural Evolution: The Case of Dog Breed Popularity](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0074770)

---

### E3-H2: "Dog ownership trends are significantly different between high-cost and low-cost real estate areas of Vienna."
Rationale: Real estate prices could affect the type of dog breeds owned, changes in real estate prices could correlate with changes in dog breeds owned. Data Needed: Dog breed data ('hunde-vie.csv'/hunde-wien.csv), Real estate prices dataset.


Author: Theresa Spiel



### E3-H3: 
* #### "Dog ownership noticeably increased during the pandemic in comparison to the previous years."
OR
* #### "Viennese dog owner tend to prefer smaller dogs instead of heavier ones."
Author: Martin Stasek

Literature / Articles: 
 * https://www.amcny.org/blog/2021/01/06/covid19-pandemic-pet-ownership/
 * https://www.zeit.de/news/2021-03/22/eine-million-mehrhaustiere-in-der-pandemie

 OR

 * https://www.petfoodindustry.com/pet-food-market/article/15449230/ownership-of-small-dogs-on-the-rise


### E3-H4: "Dog ownership is (inversely) correlated with population density in Vienna districts."
Rationale: In more densely populated areas, people might have less space for pets, leading to fewer dogs being owned. (Additionally, less densly populated districst have more green space, making it more favorable to dog ownership).
Data Needed: Dog ownership (Data1/Data2), Population density dataset. (Dog zones/greenspaces dataset)

Author: Anna Punzengruber

### E3-H5: "xxx"
Author: Group member 5
